<a href="https://colab.research.google.com/github/MahdiTheGreat/Diagnostic-Systems/blob/main/DAIS_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MahdiTheGreat/Diagnostic-Systems.git
%cd Diagnostic-Systems
%ls
!pip install optuna

Cloning into 'Diagnostic-Systems'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 92.88 KiB | 1.55 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Diagnostic-Systems
DAIS_4.ipynb  README.md  wdbc.pkl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.2 MB/s eta 0:00:00


In [18]:
!pip install -Uqq ipdb
import ipdb
%pdb on

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
Automatic pdb calling has been turned ON


In [16]:
import pickle
import pandas as pd
import numpy as np


# Load the pickle file
df = pd.read_pickle('wdbc.pkl')
df.drop('id', axis=1, inplace=True)

# Display the DataFrame
display(df[0:10])
display(df.columns)

# Basic statistics
display(df['malignant'].describe())

# Value counts (for categorical data)
display(df['malignant'].value_counts())


,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,symmetry_0,...,radius_2,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


Index(['malignant', 'radius_0', 'texture_0', 'perimeter_0', 'area_0',
       'smoothness_0', 'compactness_0', 'concavity_0', 'concave points_0',
       'symmetry_0', 'fractal dimension_0', 'radius_1', 'texture_1',
       'perimeter_1', 'area_1', 'smoothness_1', 'compactness_1', 'concavity_1',
       'concave points_1', 'symmetry_1', 'fractal dimension_1', 'radius_2',
       'texture_2', 'perimeter_2', 'area_2', 'smoothness_2', 'compactness_2',
       'concavity_2', 'concave points_2', 'symmetry_2', 'fractal dimension_2'],
      dtype='object')

,malignant
count,569.000000
mean,0.372583
std,0.483918
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


,count
malignant,
0,357
1,212


In [3]:
from sklearn.model_selection import train_test_split
target_column = 'malignant'
X = df.drop(target_column, axis=1)  # Adjust 'target' to your actual target column name
y = df[target_column]  # Adjust 'target' to your actual target column name

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_, X_val, y_test_, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [6]:
from sklearn.tree import DecisionTreeClassifier,export_text,plot_tree,export_graphviz
from sklearn.metrics import f1_score

tree=DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
print(f"f1 score: {f1_score(y_test, tree.predict(X_test)):.2f}")

f1 score: 0.93


In [7]:
import graphviz
# Print rules
tree_rules = export_text(tree, feature_names=X.columns)
print(tree_rules)
# Generate Graphviz format tree
dot_data = export_graphviz(tree,
                           feature_names=X.columns,
                           class_names=['benign',target_column],
                           filled=True, rounded=True, special_characters=True)

# Render and visualize the tree
graph = graphviz.Source(dot_data)
graph.render("decision_tree")  # Saves the tree as a file
graph.view()  # Opens the tree visualization

|--- concave points_0 <= 0.05
|   |--- radius_2 <= 16.83
|   |   |--- area_1 <= 48.70
|   |   |   |--- smoothness_2 <= 0.18
|   |   |   |   |--- smoothness_1 <= 0.00
|   |   |   |   |   |--- texture_2 <= 27.76
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- texture_2 >  27.76
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- smoothness_1 >  0.00
|   |   |   |   |   |--- texture_2 <= 33.35
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- texture_2 >  33.35
|   |   |   |   |   |   |--- texture_2 <= 33.56
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- texture_2 >  33.56
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |--- smoothness_2 >  0.18
|   |   |   |   |--- class: 1
|   |   |--- area_1 >  48.70
|   |   |   |--- concavity_1 <= 0.02
|   |   |   |   |--- class: 1
|   |   |   |--- concavity_1 >  0.02
|   |   |   |   |--- class: 0
|   |--- radius_2 >  16.83
|   |   |--- texture_0 <= 16.19
|   |   |   |--- class: 0
|   | 

'decision_tree.pdf'

In [11]:
# Example usage:
# Assuming 'df' is your DataFrame containing the features and 'y' the labels
# Compute statistics based on the labels
stats = df.groupby(y).agg(['mean', 'std'])
display(stats[0:10])

radius_0            texture_0           perimeter_0             \
                mean       std       mean       std        mean        std   
malignant                                                                    
0          12.146524  1.780512  17.914762  3.995125   78.075406  11.807438   
1          17.462830  3.203971  21.604906  3.779470  115.365377  21.854653   

               area_0             smoothness_0            ... compactness_2  \
                 mean         std         mean       std  ...          mean   
malignant                                                 ...                 
0          462.790196  134.287118     0.092478  0.013446  ...      0.182673   
1          978.376415  367.937978     0.102898  0.012608  ...      0.374824   

                    concavity_2           concave points_2            \
                std        mean       std             mean       std   
malignant                                                              
0          0.092180    0.166238  0.140368         0.074444  0.035797   
1          0.170372    0.450606  0.181507         0.182237  0.046308   

          symmetry_2           fractal dimension_2            
                mean       std                mean       std  
malignant                                                     
0           0.270246  0.041745            0.079442  0.013804  
1           0.323468  0.074685            0.091530  0.021553  

[2 rows x 60 columns]

In [31]:
from sklearn.metrics import accuracy_score, classification_report

def classify_cells_rule_based(df, stats):
    # Define thresholds based on dataset statistics

    # Larger cells indicate uncontrolled growth (malignant)
    SIZE_THRESHOLD = stats[['radius_0',]].loc[1, ('radius_0','mean')] + stats[['radius_2']].loc[1, ('radius_2','mean')]
    AREA_THRESHOLD = stats[['area_0']].loc[1, ('area_0','mean')] + stats[['area_2']].loc[1, ('area_2','mean')]
    PERIMETER_THRESHOLD = stats[['perimeter_0']].loc[1, ('perimeter_0','mean')] + stats[['perimeter_2']].loc[1, ('perimeter_2','mean')]

    WORST_SIZE_THRESHOLD = stats[['radius_1']].loc[1, ('radius_1','mean')] + stats[['radius_1']].loc[1, ('radius_1','std')]
    WORST_AREA_THRESHOLD = stats[['area_1']].loc[1, ('area_1','mean')] + stats[['area_1']].loc[1, ('area_1','std')]
    WORST_PERIMETER_THRESHOLD = stats[['perimeter_1']].loc[1, ('perimeter_1','mean')] + stats[['perimeter_1']].loc[1, ('perimeter_1','std')]


    # Malignant cells have irregular shapes and structure
    SHAPE_VARIATION_THRESHOLD = stats[['compactness_0']].loc[1, ('compactness_0','mean')] + stats[['compactness_2']].loc[1, ('compactness_2','mean')]
    SMOOTHNESS_THRESHOLD = stats[['smoothness_0']].loc[1, ('smoothness_0','mean')] + stats[['smoothness_2']].loc[1, ('smoothness_2','mean')]
    CONCAVITY_THRESHOLD = stats[['concavity_0']].loc[1, ('concavity_0','mean')] + stats[['concavity_2']].loc[1, ('concavity_2','mean')]
    CONCAVE_POINTS_THRESHOLD = stats[['concave points_0']].loc[1, ('concave points_0','mean')] + stats[['concave points_2']].loc[1, ('concave points_2','mean')]

    WORST_SHAPE_VARIATION_THRESHOLD = stats[['compactness_1']].loc[1, ('compactness_1','mean')] + stats[['compactness_1']].loc[1, ('compactness_1','std')]
    WORST_SMOOTHNESS_THRESHOLD = stats[['smoothness_1']].loc[1, ('smoothness_1','mean')] + stats[['smoothness_1']].loc[1, ('smoothness_1','std')]
    WORST_CONCAVITY_THRESHOLD = stats[['concavity_1']].loc[1, ('concavity_1','mean')] + stats[['concavity_1']].loc[1, ('concavity_1','std')]
    WORST_CONCAVE_POINTS_THRESHOLD = stats[['concave points_1']].loc[1, ('concave points_1','mean')] + stats[['concave points_1']].loc[1, ('concave points_1','std')]



    # Malignant tumors show higher texture variation
    TEXTURE_VARIATION_THRESHOLD = stats[['texture_0']].loc[1, ('texture_0','mean')] + stats[['texture_2']].loc[1, ('texture_2','mean')]

    WORST_TEXTURE_VARIATION_THRESHOLD = stats[['texture_1']].loc[1, ('texture_1','mean')] + stats[['texture_1']].loc[1, ('texture_1','std')]

    # Malignant cells tend to have asymmetric structures and higher fractal dimension
    SYMMETRY_THRESHOLD = stats[['symmetry_0']].loc[1, ('symmetry_0','mean')] + stats[['symmetry_2']].loc[1, ('symmetry_2','mean')]
    FRACTAL_DIM_THRESHOLD = stats[['fractal dimension_0']].loc[1, ('fractal dimension_0','mean')] + stats[['fractal dimension_2']].loc[1, ('fractal dimension_2','mean')]

    WORST_SYMMETRY_THRESHOLD = stats[['symmetry_1']].loc[1, ('symmetry_1','mean')] + stats[['symmetry_1']].loc[1, ('symmetry_1','std')]
    WORST_FRACTAL_DIM_THRESHOLD = stats[['fractal dimension_1']].loc[1, ('fractal dimension_1','mean')] + stats[['fractal dimension_1']].loc[1, ('fractal dimension_1','std')]


    # Classifier function based on the new thresholds
    def classify_row(row):
        for i in range(2):  # Loop through the 3 feature sets (mean, worst, standard error)
            # Get nuclear features for this set
            nuclear_radius = row[f'radius_{i}']
            nuclear_area = row[f'area_{i}']
            nuclear_perimeter = row[f'perimeter_{i}']
            compactness = row[f'compactness_{i}']
            smoothness = row[f'smoothness_{i}']
            concavity = row[f'concavity_{i}']
            concave_points = row[f'concave points_{i}']
            symmetry = row[f'symmetry_{i}']
            fractal_dimension = row[f'fractal dimension_{i}']
            texture = row[f'texture_{i}']

            size_threshold = SIZE_THRESHOLD if i == 0 else WORST_SIZE_THRESHOLD
            area_threshold = AREA_THRESHOLD if i == 0 else WORST_AREA_THRESHOLD
            perimeter_threshold = PERIMETER_THRESHOLD if i == 0 else WORST_PERIMETER_THRESHOLD
            shape_variation_threshold = SHAPE_VARIATION_THRESHOLD if i == 0 else WORST_SHAPE_VARIATION_THRESHOLD
            smoothness_threshold = SMOOTHNESS_THRESHOLD if i == 0 else WORST_SMOOTHNESS_THRESHOLD
            concavity_threshold = CONCAVITY_THRESHOLD if i == 0 else WORST_CONCAVITY_THRESHOLD
            concave_points_threshold = CONCAVE_POINTS_THRESHOLD if i == 0 else WORST_CONCAVE_POINTS_THRESHOLD
            texture_variation_threshold = TEXTURE_VARIATION_THRESHOLD if i == 0 else WORST_TEXTURE_VARIATION_THRESHOLD
            symmetry_threshold = SYMMETRY_THRESHOLD if i == 0 else WORST_SYMMETRY_THRESHOLD
            fractal_dim_threshold = FRACTAL_DIM_THRESHOLD if i == 0 else WORST_FRACTAL_DIM_THRESHOLD


            # Rule-Based Classification with refined logic
            if (nuclear_radius > size_threshold or
                nuclear_area > area_threshold or
                nuclear_perimeter > perimeter_threshold):
                return 1  # Malignant

            if (compactness > shape_variation_threshold or
                smoothness > smoothness_threshold or
                concavity > concavity_threshold or
                concave_points > concave_points_threshold):
                return 1  # Malignant

            if texture > texture_variation_threshold:
                return 1  # Malignant

            if (symmetry > symmetry_threshold or
                fractal_dimension > fractal_dim_threshold):
                return 1  # Malignant

        return 0  # Benign

    # Apply the classification function row-wise
    y_pred = df.apply(classify_row, axis=1)
    return y_pred

# Get predictions on the test set
y_pred = classify_cells_rule_based(X, stats)

# Evaluate the classifier
accuracy = accuracy_score(y, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y, y_pred))


Accuracy: 0.5817223198594025
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.68      0.67       357
           1       0.43      0.41      0.42       212

    accuracy                           0.58       569
   macro avg       0.55      0.55      0.55       569
weighted avg       0.58      0.58      0.58       569



In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import optuna


def objective(trial):
    # Define the hyperparameters to tune via Optuna
    n_estimators = trial.suggest_int('n_estimators', 50, 400)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])

    # Create the random forest classifier
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Fit the model
    clf.fit(X_train, y_train)

    # Evaluate the model
    y_pred = clf.predict(X_val)
    f1_score_value = f1_score(y_val, y_pred)

    return f1_score_value

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best f1_score_value:', study.best_trial.value)

# Best model parameters
best_params = study.best_trial.params
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
score = f1_score(y_test, y_pred)
print(f"f1 score: {score:.2f}")


[I 2025-02-18 12:56:03,115] A new study created in memory with name: no-name-5bed0d01-7897-4d18-a2cd-7fc1cca72ff7
[I 2025-02-18 12:56:04,644] Trial 0 finished with value: 0.9696969696969697 and parameters: {'n_estimators': 221, 'max_depth': 14, 'min_samples_split': 19, 'min_samples_leaf': 16, 'max_features': None}. Best is trial 0 with value: 0.9696969696969697.
[I 2025-02-18 12:56:07,419] Trial 1 finished with value: 0.9696969696969697 and parameters: {'n_estimators': 175, 'max_depth': 8, 'min_samples_split': 15, 'min_samples_leaf': 32, 'max_features': None}. Best is trial 0 with value: 0.9696969696969697.
[I 2025-02-18 12:56:08,350] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 57, 'max_depth': 20, 'min_samples_split': 27, 'min_samples_leaf': 15, 'max_features': 'log2'}. Best is trial 2 with value: 1.0.
[I 2025-02-18 12:56:10,508] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 380, 'max_depth': 7, 'min_samples_split': 42, 'min_samples_leaf': 22,

Number of finished trials: 100
Best trial: {'n_estimators': 57, 'max_depth': 20, 'min_samples_split': 27, 'min_samples_leaf': 15, 'max_features': 'log2'}
Best f1_score_value: 1.0
f1 score: 0.95


In [33]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import optuna

# Fit models and evaluate
log_reg=LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
print(f"Logistic Regression f1 score: {f1_score(y_test, log_reg.predict(X_test)):.2f}")

Logistic Regression f1 score: 0.94


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
svc=SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
score = f1_score(y_test, y_pred)
print(f"svc f1 score: {score:.2f}")


svc f1 score: 0.93
